In [1]:
import pandas as pd

df1 = pd.read_csv('CI_DMSTC_TOUR_AREA_INTRST_DGREE_INFO_20250630.csv')
df2 = pd.read_csv('YN_LDGS_PROPERTY_LC_INFO_202412.csv')
df3 = pd.read_csv('HW_LDGS_OTA_USE_LIST_AVRG_SCORE_VARTION_INFO_202412.csv')
df3_ = pd.read_csv('HW_LDGS_OTA_USE_LIST_AVRG_SCORE_VARTION_INFO_202406.csv')
df4 = pd.read_csv('YN_LDGS_STAYNG_TY_202412.csv')
df5 = pd.read_csv('HW_LDGS_DAIL_MAX_AVRG_MIN_PRC_INFO_202403-4.csv')

# 비교프레임 정제 계획
 - 1. 야놀자, 여기어때만 남기고 드롭
 - 2. 24년 전체 월 프레임 병합
 - 3. 야놀자, 여기어때 각각 숙박명별로 리뷰, 평균점수 -> 전체 평균으로 바꾸기
 - 4. 나머지 드롭
 - 5. 플랫폼별로 데이터 분리
 

## 데이터 확인

In [2]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303121 entries, 0 to 303120
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   LDGS_NM           303121 non-null  object 
 1   CTPRVN_NM         303121 non-null  object 
 2   GUGUN_NM          303121 non-null  object 
 3   OTA_NM            303121 non-null  object 
 4   BASE_YEAR         303121 non-null  int64  
 5   BASE_MT           303121 non-null  int64  
 6   BASE_DAY          303121 non-null  int64  
 7   AVRG_SCORE_VALUE  303121 non-null  float64
 8   REVIEW_REGIST_CO  303121 non-null  float64
dtypes: float64(2), int64(3), object(4)
memory usage: 20.8+ MB


In [49]:
df3['REVIEW_REGIST_CO'].value_counts()

REVIEW_REGIST_CO
1.0     95993
2.0     29362
3.0     10839
4.0      4553
5.0      1969
6.0       952
7.0       464
8.0       269
9.0       139
10.0       92
11.0       37
12.0       34
13.0       19
14.0       13
16.0       11
15.0        6
18.0        5
17.0        2
43.0        1
54.0        1
22.0        1
38.0        1
27.0        1
36.0        1
19.0        1
Name: count, dtype: int64

## 1. 야놀자, 여기어때만 남기고 전부 드롭

In [4]:
df3['OTA_NM'].value_counts()

OTA_NM
GOODCHOICE    75532
YANOLJA       69234
AGODA         66427
BOOKING       61822
HOTELS        16366
EXPEDIA       13737
INTERPARK         3
Name: count, dtype: int64

In [5]:
# goodchoice, yanolja 만 남기고 드롭
df3 = df3[df3['OTA_NM'].isin(['GOODCHOICE', 'YANOLJA'])]
df3_ = df3_[df3_['OTA_NM'].isin(['GOODCHOICE', 'YANOLJA'])]


## 데이터 병합

In [6]:
# 데이터 cancat
df = pd.concat([df3, df3_], ignore_index=True)

## 야놀자 단독 프레임 구축
- 1. 야놀자만 추출
- 2. 월별로 수치형 변수 평균값 내기
- 3. 없는 월은 작년 프레임으로 맞추기


In [7]:
df = df[df['OTA_NM'] == 'YANOLJA']
df.drop(columns=['BASE_DAY'], inplace=True)

In [8]:
# 이름별로 월마다 각각 AVRG_SCORE_VALUE,REVIEW_REGIST_CO 평균값 내기
df.groupby(['LDGS_NM', 'BASE_MT']).agg({
    'AVRG_SCORE_VALUE': 'mean',
    'REVIEW_REGIST_CO': 'mean'})
# 월별로 수치형 변수 평균값 내기
mean_df= df.groupby(['LDGS_NM', 'BASE_MT']).agg({
    'AVRG_SCORE_VALUE': 'mean',
    'REVIEW_REGIST_CO': 'mean'}).reset_index()



In [11]:

# mean_df의 수치형 변수의 평균값을 'LDGS_NM', 'BASE_MT' 별로 맵핑

df = df.merge(mean_df, on=['LDGS_NM', 'BASE_MT'], suffixes=('', '_MEAN'))
df.drop(columns=['REVIEW_REGIST_CO','AVRG_SCORE_VALUE'], inplace=True)

In [13]:
df = df.drop_duplicates(subset=['LDGS_NM', 'BASE_MT'])

In [16]:
df.columns=[['LDGS_NM', 'CTPRVN_NM', 'GUGUN_NM', 'OTA_NM', 'BASE_YEAR', 'BASE_MT',
       'AVRG_SCORE_VALUE_MEAN', 'REVIEW_REGIST_CO_MEAN',
       'AVRG_SCORE_VALUE_MEAN1', 'REVIEW_REGIST_CO_MEAN1']]

In [18]:
df.drop(columns=['AVRG_SCORE_VALUE_MEAN1','REVIEW_REGIST_CO_MEAN1'], inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_5208\1589089306.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df.drop(columns=['AVRG_SCORE_VALUE_MEAN1','REVIEW_REGIST_CO_MEAN1'], inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_5208\1589089306.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['AVRG_SCORE_VALUE_MEAN1','REVIEW_REGIST_CO_MEAN1'], inplace=True)


df

## 없는 월 채우기

- df_12 : 23년 12분기
- df_34 : 23년 34분기
- df_add : 1234분기 야놀자만 남기고 합친 데이터프레임



In [35]:
df_12 = pd.read_csv('23년1,2분기.csv')
df_34 = pd.read_csv('23년 3,4분기.csv')
df_22 = pd.read_csv('22.csv')

#### 야놀자만 남기고 전부 드롭


In [36]:
df_12 = df_12[df_12['OTA_NM'] == 'YANOLJA']
df_12.drop(columns=['BASE_DAY'], inplace=True)
df_34 = df_34[df_34['OTA_NM'] == 'YANOLJA']
df_34.drop(columns=['BASE_DAY'], inplace=True)
df_22 = df_22[df_22['OTA_NM'] == 'YANOLJA']
df_22.drop(columns=['BASE_DAY'], inplace=True)

In [37]:
df_add = pd.concat([df_12, df_34,df_22], ignore_index=True)

In [41]:
df_add['BASE_MT'].value_counts()

BASE_MT
8     1241
7     1138
10     908
9      900
12     458
11     423
1      319
2      257
3      208
4      139
5      112
6       77
Name: count, dtype: int64

In [42]:
# 이름별로 월마다 각각 AVRG_SCORE_VALUE,REVIEW_REGIST_CO 평균값 내기
df_add.groupby(['LDGS_NM', 'BASE_MT']).agg({
    'AVRG_SCORE_VALUE': 'mean',
    'REVIEW_REGIST_CO': 'mean'})
# 월별로 수치형 변수 평균값 내기
mean_df_add= df_add.groupby(['LDGS_NM', 'BASE_MT']).agg({
    'AVRG_SCORE_VALUE': 'mean',
    'REVIEW_REGIST_CO': 'mean'}).reset_index()


In [43]:

# mean_df의 수치형 변수의 평균값을 'LDGS_NM', 'BASE_MT' 별로 맵핑

df_add = df_add.merge(mean_df_add, on=['LDGS_NM', 'BASE_MT'], suffixes=('', '_MEAN'))
df_add.drop(columns=['REVIEW_REGIST_CO','AVRG_SCORE_VALUE'], inplace=True)

In [44]:
df_add

,LDGS_NM,CTPRVN_NM,GUGUN_NM,OTA_NM,BASE_YEAR,BASE_MT,AVRG_SCORE_VALUE_MEAN,REVIEW_REGIST_CO_MEAN
0,G2 호텔,서울,중구,YANOLJA,2023,1,9.333333,1.000000
1,G2 호텔,서울,중구,YANOLJA,2023,1,9.333333,1.000000
2,G2 호텔,서울,중구,YANOLJA,2023,1,9.333333,1.000000
3,SR 호텔 사당,서울,동작구,YANOLJA,2023,1,9.857143,1.571429
4,SR 호텔 사당,서울,동작구,YANOLJA,2023,1,9.857143,1.571429
...,...,...,...,...,...,...,...,...
6175,씨엘오션호텔,부산,강서구,YANOLJA,2022,10,9.550000,1.700000
6176,씨엘오션호텔,부산,강서구,YANOLJA,2022,10,9.550000,1.700000
6177,씨엘오션호텔,부산,강서구,YANOLJA,2022,10,9.550000,1.700000
6178,씨엘오션호텔,부산,강서구,YANOLJA,2022,10,9.550000,1.700000


In [46]:
df

,LDGS_NM,CTPRVN_NM,GUGUN_NM,OTA_NM,BASE_YEAR,BASE_MT,AVRG_SCORE_VALUE_MEAN,REVIEW_REGIST_CO_MEAN
0,솔라리아 니시테츠 부산,부산광역시,부산진구,YANOLJA,2024,6,10.0,1.25
4,솔라리아 니시테츠 부산,부산광역시,부산진구,YANOLJA,2024,7,9.2,1.20
9,부산 비즈니스 호텔,부산광역시,부산진구,YANOLJA,2024,6,8.9,1.60
19,부산 비즈니스 호텔,부산광역시,부산진구,YANOLJA,2024,7,10.0,1.00
20,스탠포드 인 부산,부산광역시,중구,YANOLJA,2024,6,10.0,1.00
...,...,...,...,...,...,...,...,...
116535,광주 충장로 하운드 호텔(구. 워싱턴 호텔),광주광역시,동구,YANOLJA,2024,2,9.2,1.00
116540,광주 충장로 하운드 호텔(구. 워싱턴 호텔),광주광역시,동구,YANOLJA,2024,3,9.5,1.25
116548,광주 충장로 하운드 호텔(구. 워싱턴 호텔),광주광역시,동구,YANOLJA,2024,4,9.6,1.50
116558,광주 충장로 하운드 호텔(구. 워싱턴 호텔),광주광역시,동구,YANOLJA,2024,5,8.0,1.50


In [48]:
df[df['LDGS_NM']=='SR 호텔 사당']

,LDGS_NM,CTPRVN_NM,GUGUN_NM,OTA_NM,BASE_YEAR,BASE_MT,AVRG_SCORE_VALUE_MEAN,REVIEW_REGIST_CO_MEAN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
116535,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116540,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116548,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
df['LDGS_NM'].value_counts(
    
)

(LDGS_NM,)   
3s 부티크 호텔        11
H-AVENUE 광안리점    11
JOY 모텔           11
황남관 한옥체험마을       11
WD호텔             11
                 ..
제천 송계계곡캠핑장펜션      1
제천 산중다연펜션         1
제천 산과계곡그리고나펜션     1
제천 명암오토캠핑장펜션      1
제천 호수민박펜션         1
Name: count, Length: 3653, dtype: int64

# 정제 계획
    **숙소 id별로 필요한 컬럼 매핑 후 필요한 열 웹크롤링 진행**
- 1. 숙소id 

In [ ]:
import pandas as pd
df = pd.read_csv('YN_LDGS_STAYNG_TY_202412.csv')
df1 =pd.read_csv('YN_LDGS_PROPERTY_INFO_202412.csv')
df2 =pd.read_csv('YN_LDGS_PROPERTY_LC_INFO_202412.csv')
df3 =pd.read_csv('YN_LDGS_PROPERTY_USGTM_202412.csv')

In [ ]:
# df1, df2, df3을 LDGS_CD, LDGS_NM 기준으로 df에 매핑(merge)
df_merged = df.merge(df1, on=['LDGS_CD', 'LDGS_NM'], how='left') \
              .merge(df2, on=['LDGS_CD', 'LDGS_NM'], how='left') \
              .merge(df3, on=['LDGS_CD', 'LDGS_NM'], how='left')
df_merged

In [ ]:
누락값 = df_merged[df_merged['CHIN_TIME'].isna()]

In [ ]:
누락값.to_csv('체크인아웃누락.csv')

In [ ]:
df_nan = pd.read_csv('YN_LDGS_PROPERTY_USGTM_202411.csv')

In [ ]:
df_nan1 = pd.read_csv('YN_LDGS_PROPERTY_USGTM_202410.csv')

In [ ]:
df_nan1

In [ ]:
df_merged

In [ ]:
df_merged.to_csv('메인프레임.csv', index=False)

In [ ]:
df_merged = pd.read_csv('메인프레임.csv')

In [ ]:
df_merged['LDGS_NM'].value_counts(
    
)

In [ ]:
df_merged['LDGS_NM'] = df_merged['LDGS_NM'].str.replace(' ', '')

# 검색어 매핑

In [ ]:
searchdf = pd.read_csv('검색량.csv')

In [ ]:
searchdf['숙소'].value_counts(
    
)

In [ ]:

keyword= searchdf.set_index('숙소')['월간 검색량(전체)'].to_dict()

df_merged['숙소 검색량(전체)'] = df_merged['LDGS_NM'].map(keyword)



- 누락값 확인

In [ ]:
searchdf_re = df_merged[df_merged['숙소 검색량(전체)'].isna()]

In [ ]:
searchdf_re.to_csv('searchdf_re.csv', index=False)

In [ ]:
df= pd.read_csv('searchdf_re.csv')

- 확인 후 재 웹크롤링

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
# 1. 키워드 불러오기
df = pd.read_csv('searchdf_re.csv')
sample = df.iloc[:, 1].dropna().astype(str).str.strip().tolist()
# 2. 100개씩 분할
def chunk_list(lst, size):
    return [lst[i:i+size] for i in range(0, len(lst), size)]
batches = chunk_list(sample, 100)
# 3. 드라이버 설정 및 실행
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)
options.add_argument("--disable-blink-features=AutomationControlled")
driver = webdriver.Chrome(options=options)
driver.get("https://keywordsound.com/service/keyword-analysis-bulk")

In [ ]:
# input(":오른쪽을_가리키는_손_모양: 로그인 후 Enter 키를 누르세요...")
# 4. 크롤링
results = []
for idx, group in enumerate(batches):
    print(f"\n:돋보기: :앞쪽_화살표: {idx+1}번째 그룹 ({len(group)}개 키워드) 조회 중...")
    textarea = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="inputKeywordsBulk"]'))
    )
    driver.execute_script("arguments[0].click();", textarea)
    textarea.send_keys(Keys.CONTROL, 'a')
    textarea.send_keys(Keys.DELETE)
    time.sleep(1.5)
    textarea.send_keys('\n'.join(group))
    time.sleep(1.5)
    # 조회 버튼 클릭
    try:
        button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="btnKeywordAnalysisBulk"]'))
        )
        driver.execute_script("arguments[0].click();", button)
    except Exception as e:
        print(f":x: 조회 클릭 실패: {type(e).__name__} / {str(e)}")
    # 결과 로딩 대기
    WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.XPATH, "//tbody/tr")))
    time.sleep(2)
    # 결과 수집
    row_count = len(driver.find_elements(By.XPATH, "//tbody/tr"))
    for i in range(row_count):
        try:
            row = driver.find_element(By.XPATH, f"//tbody/tr[{i+1}]")
            tds = row.find_elements(By.TAG_NAME, "td")
            if len(tds) >= 5:
                keyword_text = tds[1].text.strip()
                volume_text = tds[4].text.strip()
                results.append({"숙소": keyword_text, "월간 검색량(전체)": volume_text})
                print(f"{len(results):>4} :앞쪽_화살표: {keyword_text}: {volume_text}")
        except:
            continue
    # 삭제 체크박스 클릭 후 삭제 버튼 클릭
    try:
        checkbox = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="tableAnalysisBulk_wrapper"]/div[2]/div/div[1]/div/table/thead/tr[1]/th[1]/label/input'))
        )
        driver.execute_script("arguments[0].click();", checkbox)
        time.sleep(1)
        delete_btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="tableAnalysisBulk_wrapper"]/div[1]/div[2]/div/button[1]'))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", delete_btn)
        driver.execute_script("arguments[0].click();", delete_btn)
        time.sleep(1)
    except Exception as e:
        print(f":경고: 삭제 실패: {type(e).__name__} / {str(e)}")
    time.sleep(4)
# 5. 결과 저장
df_result_re = pd.DataFrame(results)

print(f"\n:흰색_확인_표시: 전체 키워드 수집 완료: {len(df_result_re)}건")
print(df_result_re.head())
# 6. 누락 키워드 검출
crawled_keywords = df_result_re['숙소'].dropna().unique()
missing = pd.Series(sample)[~pd.Series(sample).isin(crawled_keywords)]
print(f"\n:x: 누락 키워드 수: {len(missing)}개")
if not missing.empty:
    print("예시 :앞쪽_화살표:", missing.head(10).tolist())
# 7. 종료
driver.quit()# input(":오른쪽을_가리키는_손_모양: 로그인 후 Enter 키를 누르세요...")


In [ ]:
df_result_re[df_result_re['월간 검색량(전체)']=='12,970']

In [ ]:
searchdf_re[searchdf_re['LDGS_NM']=='제부도궁평항씨사이드']

In [ ]:
df_result_re.info()

### 매핑코드

In [ ]:

keyword= df_result_re.set_index('숙소')['월간 검색량(전체)'].to_dict()

searchdf_re['숙소 검색량(전체)'] = searchdf_re['LDGS_NM'].map(keyword)

In [ ]:
searchdf_re

- 재크롤링한 결과df : df_result_re
- 메인 프레임: df_merged

### 재크롤링 결과 :  누락값은 거의 10개  미만 제외한 나머지 값은 20임 fiilna로 누락값 대체

In [ ]:
df_merged['숙소 검색량(전체)'] = df_merged['숙소 검색량(전체)'].fillna(20)

In [ ]:
df_merged['숙소 검색량(전체)'].value_counts()

- 여유시간 대 수기로 조정

# 시도명 정제

In [ ]:
df_merged[df_merged['LDGS_ROAD_NM_ADDR'].str.contains('대전광역시')]


In [ ]:
df_merged_시도 = df_merged.copy()

In [ ]:
df_merged_시도[['상위 시도', '하위 시구']] = df_merged_시도['LDGS_ROAD_NM_ADDR'].str.split().str[:2].apply(pd.Series)

In [ ]:
# 상위 시도 정제 매핑
sido_clean_map = {
    '서울특별시': '서울', '서울시': '서울', '서울특별시중구': '서울',
    '부산광역시': '부산', '부산시': '부산',
    '대구광역시': '대구', '대구시': '대구',
    '인천광역시': '인천', '인천시': '인천',
    '광주광역시': '광주',
    '대전광역시': '대전', '대전시': '대전',
    '울산광역시': '울산', '울산시': '울산',
    '세종특별자치시': '세종',
    '경기도': '경기', '가평군': '경기',
    '강원도': '강원', '강원특별자치도': '강원', '강원도강릉시': '강원', '양양군': '강원', '홍천군': '강원',
    '충청북도': '충북', '충북': '충북',
    '충청남도': '충남',
    '전라북도': '전북', '전북특별자치도': '전북', '전주시': '전북',
    '전라남도': '전남', '담양군': '전남',
    '경상북도': '경북', '경주시': '경북',
    '경상남도': '경남', '밀양시': '경남',
    '제주특별자치도': '제주', '제주시': '제주', '서귀포시': '제주',
    '대한민국': '제주'  # 이상치로 제거하거나 NaN 처리
}

In [ ]:
df_merged_시도['상위 시도'] = df_merged_시도['상위 시도'].map(lambda x: sido_clean_map.get(x, x))

- 대한민국이라고 되어 있는 행은 제주도인것을 파악하고 수기로 정제

In [ ]:
df_merged_시도['상위 시도'].value_counts()

### 강화군 확인 인천으로 정제

In [ ]:
df_merged_시도[df_merged_시도['상위 시도']=='강화군']

In [ ]:
df_merged_시도.loc[25030,['상위 시도']]='인천'

## 상위 시도 기준으로 지역명 관심도 매핑

In [ ]:
관심도 = pd.read_csv('지역별_관심도_최종 (1).csv')

In [ ]:
관심도

### 매핑

In [ ]:

keyword= 관심도.set_index('지역')['지역별_관심도'].to_dict()

df_merged_시도['지역별_관심도'] = df_merged_시도['상위 시도'].map(keyword)

In [ ]:
df_merged_시도['상위 시도'].value_counts()

In [ ]:
df_merged_시도.info()

### 누락확인

In [ ]:
df_merged_시도[df_merged_시도['상위 시도'].isna()]

In [ ]:
df = df_merged_시도.copy()

# 하위시구 정제

In [ ]:
print(df['하위 시구'].value_counts(dropna=False))

In [ ]:
df[df['하위 시구'].isna()]

In [ ]:
df.loc[26440,['하위 시구']]='제주시'

In [ ]:
df.to_csv('0729_메인프레임v2.csv', index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('0729_메인프레임v2.csv')
df

# 유튜브 평균 조회수 매핑

In [ ]:
ytdf = pd.read_csv('지역별_유튜브_평균조회수.csv')
ytdf

In [ ]:

keyword= ytdf.set_index('지역')['유튜브_상위20_평균조회수'].to_dict()

df['유튜브_상위20_평균조회수'] = df['상위 시도'].map(keyword)
df.to_csv('0731_메인프레임.v3.csv', index=False)

# 가장 오래된 리뷰 날짜 매핑 

In [ ]:
import pandas as pd
df1 = pd.read_excel('yanolja_oldest_reviews_1.xlsx')
df2 = pd.read_excel('yanolja_oldest_reviews_2.xlsx')
df3 = pd.read_excel('yanolja_oldest_reviews_3.xlsx')
df4 = pd.read_excel('yanolja_oldest_reviews_4.xlsx')
df5 = pd.read_excel('yanolja_oldest_reviews_5.xlsx')
df6 = pd.read_excel('7002~12999yanolja_oldest_reviews_6.xlsx')
df7 = pd.read_excel('초반1.xlsx')
df8 = pd.read_excel('초반2.xlsx')
df9 = pd.read_excel('초반3.xlsx')
df10 = pd.read_excel('초반4.xlsx')
df11 = pd.read_excel('초반5.xlsx')
df12 = pd.read_excel('초반6.xlsx')
df13 = pd.read_excel('후반1.xlsx')
df14 = pd.read_excel('후반2.xlsx')
df15 = pd.read_excel('후반3.xlsx')
df16 = pd.read_excel('후반4.xlsx')
df17 = pd.read_excel('후반5.xlsx')
df18 = pd.read_excel('후반6.xlsx')
df19 = pd.read_excel('yanolja_oldest_reviews_7.xlsx')
df20 = pd.read_excel('yanolja_oldest_reviews_8.xlsx')
df21 = pd.read_excel('yanolja_oldest_reviews_9.xlsx')
df22 = pd.read_excel('yanolja_oldest_reviews_10.xlsx')
df23 = pd.read_excel('yanolja_oldest_reviews_11.xlsx')
df24 = pd.read_excel('yanolja_oldest_reviews_12.xlsx')
df25 = pd.read_excel('yanolja_oldest_reviews_13.xlsx')
df26 = pd.read_excel('yanolja_oldest_reviews_14.xlsx')

- 추출된 데이터 프레임 병합

In [ ]:

df = pd.concat([df1, df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df21,df22,df23,df24,df25,df26], axis=0, ignore_index=True)

In [ ]:
df.info()

In [ ]:
df_main = pd.read_csv('0731_메인프레임.v3.csv')

In [ ]:
df_main['LDGS_CD'] = df_main['LDGS_CD'].astype(int)

In [ ]:

date= df.set_index('LDGS_CD')['oldest_review_date'].to_dict()

df_main['가장 오래된 리뷰'] = df_main['LDGS_CD'].map(date)

error = df.set_index('LDGS_CD')['error'].to_dict()
df_main['error'] = df_main['LDGS_CD'].map(error)

In [ ]:
df_main.info()

In [ ]:
df_main.to_csv('0805_메인프레임.v4.csv',index=False)

In [ ]:
df_nu = df_main[df_main['가장 오래된 리뷰'].isnull() & df['error'].isnull()]
df_nu.info()

In [ ]:
import pandas as pd

# CSV 파일을 불러와 df_main 데이터프레임으로 저장합니다.
df_main = pd.read_csv('0805_메인프레임.v4.csv')

# '가장 오래된 리뷰'와 'error' 열에 모두 누락된 값이 있는 행을 찾아 새로운 데이터프레임(df_nu)에 저장합니다.
# .isnull() 메서드는 누락된 값(NaN)이 있으면 True를 반환하며, '&' 연산자는 두 조건이 모두 True일 때만 해당 행을 선택합니다.
df_nu = df_main[df_main['가장 오래된 리뷰'].isnull() & df_main['error'].isnull()]

# df_nu 데이터프레임의 정보를 출력합니다.
df_nu

In [ ]:
df_ = pd.read_excel('last_yanolja_oldest_reviews_1.xlsx')
df_1 = pd.read_excel('last_yanolja_oldest_reviews_2.xlsx')
df_2 = pd.read_excel('lastddd_yanolja_oldest_reviews_3.xlsx')

In [ ]:
dflast = pd.concat([df_,df_1,df_2], axis=0, ignore_index=True)

In [ ]:
dflast.to_csv('lastcrolling.csv',index=False)

In [ ]:
dflast

In [ ]:
import pandas as pd

# 파일 불러오기
main_df = pd.read_csv("0805_메인프레임.v4.csv")
last_df = pd.read_csv("lastcrolling.csv")



In [ ]:
# 'LDGS_CD' 기준으로 left join 수행
merged_df = main_df.merge(last_df, on='LDGS_CD', how='left', suffixes=('', '_new'))

# '가장 오래된 리뷰', 'error' 값이 누락된 경우에만 새로운 값으로 채움
merged_df['가장 오래된 리뷰'] = merged_df['가장 오래된 리뷰'].fillna(merged_df['oldest_review_date'])
merged_df['error'] = merged_df['error'].fillna(merged_df['error_new'])

# 임시로 생성된 열 제거
merged_df = merged_df.drop(columns=['oldest_review_date', 'error_new'])

merged_df.to_csv('0805_메인프레임v5.csv',index=False)

In [ ]:
merged_df[merged_df['가장 오래된 리뷰'].isna()]

In [ ]:
import pandas as pd
djf = pd.read_csv('0805_메인프레임v5.csv')

djf.info()

In [ ]:
import pandas as pd
df = pd.read_excel('re_oldest_reviews_1.xlsx')
df1 = pd.read_excel('re_oldest_reviews_2.xlsx')
df2 = pd.read_excel('re_oldest_reviews_3.xlsx')
df3 = pd.read_excel('다시1.xlsx')
df4 = pd.read_excel('다시2.xlsx')
df5 = pd.read_excel('다시3.xlsx')

In [ ]:
dflast = pd.concat([df,df1,df2,df3,df4,df5], axis=0, ignore_index=True)

In [ ]:
dflast.to_csv('5000review_0807.csv',index=False)

In [ ]:
# 새로 업로드된 파일 불러오기 (5000개 업데이트용)
update_df = pd.read_csv("5000review_0807.csv")

# 컬럼 확인
update_df.columns


In [ ]:
import pandas as pd

# 파일 경로
main_path = '0805_메인프레임v5.csv'
review_path = '5000review_0807.csv'

# CSV 파일 읽기
df_main = pd.read_csv(main_path)
df_review = pd.read_csv(review_path)


# 필요한 열만 남기고 정제 (LDGS_CD, 가장 오래된 리뷰)
df_review_filtered = df_review[['LDGS_CD', 'oldest_review_date']]

# 'LDGS_CD' 기준으로 merge (left join)
df_merged = df_main.merge(df_review_filtered, on='LDGS_CD', how='left', suffixes=('', '_추가'))

# 기존 '가장 오래된 리뷰'가 누락된 경우에만 새 값으로 채우기
df_merged['가장 오래된 리뷰'] = df_merged['가장 오래된 리뷰'].combine_first(df_merged['oldest_review_date_추가'])

# 보조 열 삭제
df_merged.drop(columns=['가장 오래된 리뷰_추가'], inplace=True)

In [ ]:
import pandas as pd

# 파일 경로
main_path = '0805_메인프레임v5.csv'
review_path = '5000review_0807.csv'

# CSV 파일 읽기
df_main = pd.read_csv(main_path)
df_review = pd.read_csv(review_path)

# 필요한 열만 남기고 정제 (LDGS_CD, oldest_review_date)
df_review_filtered = df_review[['LDGS_CD', 'oldest_review_date']]

# 'LDGS_CD' 기준으로 merge (left join)
# suffixes를 사용하면 이름이 중복되는 열에만 적용되므로,
# 현재 코드에서는 'oldest_review_date_추가'가 생성되지 않습니다.
df_merged = df_main.merge(df_review_filtered, on='LDGS_CD', how='left')

# 기존 '가장 오래된 리뷰'가 누락된 경우에만 새 값으로 채우기
df_merged['가장 오래된 리뷰'] = df_merged['가장 오래된 리뷰'].combine_first(df_merged['oldest_review_date'])

# 보조 열 삭제
# merge 후 생성된 열의 이름은 'oldest_review_date'이므로, 해당 열을 삭제해야 합니다.
df_merged.drop(columns=['oldest_review_date'], inplace=True)

In [ ]:
df_merged.info()

In [ ]:
import pandas as pd

# 파일 경로
main_path = '0805_메인프레임v5.csv'
review_path = '5000review_0807.csv'

# CSV 파일 읽기
df_main = pd.read_csv(main_path)
df_review = pd.read_csv(review_path)

# 필요한 열만 남기고 정제 (LDGS_CD, oldest_review_date)
df_review_filtered = df_review[['LDGS_CD', 'oldest_review_date']]

# 'LDGS_CD' 기준으로 merge (left join)
# 기존 '가장 오래된 리뷰'와 충돌하는 열이 없으므로 suffixes를 지정할 필요가 없습니다.
# 하지만, suffixes를 사용한다면 병합된 열 이름은 'oldest_review_date_추가'가 됩니다.
df_merged = df_main.merge(df_review_filtered, on='LDGS_CD', how='left', suffixes=('', '_추가'))

# 기존 '가장 오래된 리뷰'가 누락된 경우에만 새 값으로 채우기
# combine_first() 메서드는 첫 번째 시리즈의 NaN 값을 두 번째 시리즈의 값으로 채웁니다.
df_merged['가장 오래된 리뷰'] = df_merged['가장 오래된 리뷰'].combine_first(df_merged['oldest_review_date_추가'])

# 보조 열 삭제
# merge로 생성된 임시 열의 이름은 'oldest_review_date_추가'입니다.
df_merged.drop(columns=['oldest_review_date_추가'], inplace=True)

In [ ]:
# 'LDGS_CD' 기준으로 left join 수행
merged_df = main_df.merge(dflast, on='LDGS_CD', how='left', suffixes=('', '_new'))

# '가장 오래된 리뷰', 'error' 값이 누락된 경우에만 새로운 값으로 채움
merged_df['가장 오래된 리뷰'] = merged_df['가장 오래된 리뷰'].fillna(merged_df['oldest_review_date'])
merged_df['error'] = merged_df['error'].fillna(merged_df['error_new'])

# 임시로 생성된 열 제거
merged_df = merged_df.drop(columns=['oldest_review_date', 'error_new'])

merged_df.to_csv('0807_메인프레임v6.csv',index=False)

In [ ]:
import pandas as pd

# 1. 파일 불러오기
# 기존 메인 데이터프레임 (27,008개 행)을 불러옵니다.
main_df = pd.read_csv("0805_메인프레임v5.csv")
# 새로 수집한 5,000개 리뷰 정보가 담긴 데이터프레임을 불러옵니다.
dflast = pd.read_csv('5000review_0807.csv')

# 2. 'LDGS_CD' 기준으로 데이터 병합
# left join을 사용하여 main_df의 모든 행을 유지하고, dflast의 일치하는 행 정보를 병합합니다.
# 이 단계에서는 행의 개수가 main_df와 동일하게 27,008개로 유지됩니다.
merged_df = main_df.merge(dflast, on='LDGS_CD', how='left', suffixes=('', '_new'))

# 3. 누락된 값(NaN) 채우기
# 기존 '가장 오래된 리뷰' 열에 값이 없는 경우, 새로 병합된 'oldest_review_date' 열의 값으로 채웁니다.
# 'error' 열도 같은 방식으로 업데이트합니다.
merged_df['가장 오래된 리뷰'] = merged_df['가장 오래된 리뷰'].fillna(merged_df['oldest_review_date'])
merged_df['error'] = merged_df['error'].fillna(merged_df['error_new'])

# 4. 불필요한 임시 열 삭제
# 병합 과정에서 생성된 임시 열을 삭제하여 최종 데이터프레임을 정리합니다.
merged_df = merged_df.drop(columns=['oldest_review_date', 'error_new'])

# 5. 최종 데이터프레임 저장
# 업데이트가 완료된 데이터프레임을 '0807_메인프레임v6.csv' 파일로 저장합니다.
#merged_df.to_csv('0807_메인프레임v6.csv', index=False)

# 최종 결과 확인 (선택 사항)
print(merged_df.info())

In [ ]:
merged_df.info()

In [ ]:
import pandas as pd
df0808 = pd.read_csv('0808_메인프레임v8.csv')
df0808.info()

In [ ]:
df0807 = pd.read_csv('0808_메인프레임v7.csv')
df0807.info()

In [ ]:
df0807['상위 시도'].value_counts()

In [ ]:
import pandas as pd
df_ver = pd.read_csv('vertex_image_analysis_results.csv', encoding='utf-8-sig')
df_score = pd.read_csv('야놀자_감성예측완료.csv', encoding='utf-8-sig')
df_main = pd.read_csv('0814_메인프레임v13.csv')


In [ ]:
df_main.info()

In [ ]:
df_score = df_score.drop(['동반유형','리뷰내용','작성일'],axis=1)

In [ ]:
df_score = df_score.drop('별점',axis=1)

In [ ]:
df_score.to_csv('추천시스템용_감성분석.csv', index=False)

In [ ]:
df_score

In [ ]:
import pandas as pd

# 감성분석 데이터프레임 로드
df_sentiment = pd.read_csv('추천시스템용_감성분석.csv')

# 이미지 분석 결과 데이터프레임 로드
df_image_analysis = pd.read_csv('vertex_image_analysis_results.csv')

# 숙소ID와 file_id 컬럼 확인
print(df_sentiment.columns)
print(df_image_analysis.columns)
filtered_sentiment = df_sentiment[df_sentiment['숙소ID'].isin(df_image_analysis['file_id'])]
filtered_sentiment.info()

In [ ]:
filtered_sentiment.to_csv('추천 시스템용.csv', index=False)

In [ ]:
df_ver[df_ver['file_id']==23197]

# 클러스터링용 데이터프레임 정제

In [ ]:
import pandas as pd
df_main = pd.read_csv('0822_메인프레임v13.csv')
df_score = pd.read_csv('숙소별_감성별점_평균.csv')
df_score

In [ ]:
df_main

In [ ]:

keyword= df_score.set_index('숙소ID')['감성별점'].to_dict()

df_main['감성별점_평균'] = df_main['숙소ID'].map(keyword)

- 누락 확인

In [ ]:
df_main[df_main['감성별점_평균'].isna()]

In [ ]:
df_main.to_csv('클러스터링용_프레임.csv', index=False )